# OpenRouteService python bindings

https://github.com/GIScience/openrouteservice-py

In [14]:
import openrouteservice as ors
import folium
import json
import random
from pprint import pprint

url_ors = 'http://ors:8080/ors'
url_vroom = 'http://vroom:8081'

pointsA = json.load(open('./data/mezzolombardo.geojson'))['features'] #4 points
pointsB = json.load(open('./data/mezzocorona.geojson'))['features'] #6 points

#Povo [46.0682, 11.1551]
center = [46.2147, 11.1065]
zoom = 15

#s = 0.005 #impostare a 0 se partenza e' uguale a destinazione
s = 0

point_start = [46.2170,11.1066]
point_end = [point_start[0], point_start[1] - s]


# OpenRouteService TSP Routing example

example of one vehicle for two groups of bins all full, without considering the capacity of vehicle.

- [API Directions](https://openrouteservice.org/dev/#/api-docs/v2/directions/{profile}/post)
- [Preferences examples](https://github.com/GIScience/openrouteservice-docs#examples)
- [more examples](https://github.com/GIScience/openrouteservice-py/blob/master/examples/basic_example.ipynb)

In [22]:
M = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)
    
client = ors.Client(base_url=url_ors, key='')

ccA = [p['geometry']['coordinates'] for p in pointsA]
ccB = [p['geometry']['coordinates'] for p in pointsB]


#include partenza e arrivo
dirccA = [list(reversed(point_start))] + ccA + [list(reversed(point_end))]
dirccB = [list(reversed(point_end))] + ccB + [list(reversed(point_end))]

routeA = client.directions(
    coordinates = dirccA,
    profile = 'driving-car',
    format = 'geojson',
    preference = 'fastest',#shortest
    #restrictions =  ... width,height(in metri), weight(in tonnellate)
    validate = False
)

routeB = client.directions(
    coordinates = dirccB,
    profile = 'driving-car',
    format = 'geojson',
    validate = False
)

polyA = [list(reversed(coord)) for coord in routeA['features'][0]['geometry']['coordinates']]
polyB = [list(reversed(coord)) for coord in routeB['features'][0]['geometry']['coordinates']]

folium.PolyLine(polyA, color='red').add_to(M)
folium.PolyLine(polyB, color='blue').add_to(M)

for c in (ccA + ccB):
    loc = list(reversed(c))
    folium.Marker(loc, icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(M)

folium.Marker(point_start, icon=folium.Icon(icon='truck', prefix='fa', color='blue')).add_to(M)
    
M

## VRP Vehicle Routing Problem 

Optimize job scheduling for multiple vehicles.
The next example makes more use of the power of the ORS optimization endpoint(using Vroom)
- [API Optimization](https://openrouteservice.org/dev/#/api-docs/optimization/post)
- [API VROOM](https://github.com/VROOM-Project/vroom/blob/master/docs/API.md)

**Definition of 2 vehicles each having capacity 5 and bins having contents a random value from 1 to 3**

In [26]:
m = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

vehicles = []

# Veicolo B blu
capacityB = 5
vehicles.append(ors.optimization.Vehicle(
    id = 1,
    capacity = [capacityB],  # Limite di capacita' del Veicolo    
    profile = 'driving-hgv',
    
    start = list(reversed(point_start)), # posizione di partenza veicoli
    end = list(reversed(point_end)),
))
folium.Marker(point_start, tooltip=folium.Tooltip(capacityB,permanent=True), icon=folium.Icon(icon='truck', prefix='fa', color='blue')).add_to(m)

# Veicolo A rosso
point_start2 = [point_start[0] + 0.0015, point_start[1] - 0.0015]
capacityA = 5
vehicles.append(ors.optimization.Vehicle(
    id = 1,
    capacity = [capacityA],  # Limite di capacita' del Veicolo    
    profile = 'driving-hgv',
    #restrictions =  ... width,height(in metri), weight(in tonnellate)
    start = list(reversed(point_start2)), # posizione di partenza veicoli
    end = list(reversed(point_start2)),
))
folium.Marker(point_start2, tooltip=folium.Tooltip(capacityA,permanent=True), icon=folium.Icon(icon='truck', prefix='fa', color='red')).add_to(m)

#Bidoni 4 + 6 totali 10
job_locs = ccA + ccB

jobs = []
for idx, cc in enumerate(job_locs):
    loc = list(reversed(cc))
    amount = random.randrange(1, 5)
    jobs.append(ors.optimization.Job(
        id = idx,
        location = cc,
        #priority = [1-100]
        #amount = [2] # Quantita' contenuto nel Bidone
        amount = [amount],
        #priority = potrebbe essere utile anche priority
        #ma non e' supportato, bug che verra presto risolto https://github.com/GIScience/openrouteservice-py/issues/39
    ))
    folium.Marker(loc, tooltip=folium.Tooltip(amount,permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(m)

m

In [27]:
clientVrp = ors.Client(base_url=url_vroom, key='')

optimized = clientVrp.optimization(
    vehicles = vehicles,
    jobs = jobs,
    geometry = True
)

routeA = optimized['routes'][0]
routeB = optimized['routes'][1]

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(routeA['geometry'])['coordinates']],
    color='blue'
).add_to(m)

folium.PolyLine(
    locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(routeB['geometry'])['coordinates']],
    color='red'
).add_to(m)

m

In [27]:
vehicle1 = optimized['routes'][0]

vehicle2 = optimized['routes'][1]

pprint(vehicle1)


{'amount': [5],
 'cost': 350,
 'distance': 3431,
 'duration': 350,
 'geometry': 'iwayGggxbADe@HqAA{@AwA@cAPwCNqBVmBRuANs@Jk@p@uCr@gCn@eBgDVqAFeCCIHGZF[HIdCBpAGfDW~@kBd@}@Rw@BKNkDDYJk@Ju@Do@@W?O?M?MIeCGgAAk@Ca@?GA]AcA?E?K?WFSFCDEBKRVJ^RhAJZPTr@d@B@r@TbA^n@PZHNBtBd@v@J^DJb@DFf@x@Xb@Vb@p@r@b@TLF|DnBDBhExBFBILaDdFkGjKuArAqAlAmCnCMJ[XMJyArAkG`G]Nc@\\MDw@Ke@@iCx@O@KGm@uAS_@c@c@@vA@z@IpAEd@',
 'service': 0,
 'steps': [{'arrival': 0,
            'distance': 0,
            'duration': 0,
            'location': [11.1066, 46.217],
            'type': 'start'},
           {'arrival': 72,
            'distance': 758,
            'duration': 72,
            'job': 4,
            'location': [11.112627983093262, 46.21758452155989],
            'service': 0,
            'type': 'job',
            'waiting_time': 0},
           {'arrival': 136,
            'distance': 1419,
            'duration': 136,
            'job': 5,
            'location': [11.118206977844238, 46.214911991592125],
            

In [42]:
pprint(vehicle2)

{'amount': [5],
 'cost': 588,
 'distance': 4436,
 'duration': 588,
 'geometry': 'cabyGe}wbAbAfB_DrFk@hAOb@@x@JVjBlEDJBFbFxLHRf@lAjAhCPb@@DBDTl@Nn@Hp@Hf@Aj@A|@An@?d@Av@EtE?R?^AdABz@Hn@BPXnAR|@c@VEB_@V{@A}@FOFNG|@Gz@@^WDCb@W\\SZSlAs@JCP?f@HLDh@XBBHFNJDHNVJNFLJRDFp@jA@@NVXf@BDFHp@lAJPRZJR[\\STeA|@EFIDURe@^oAnAML}@~@u@v@cA`AWXQRGFCBBCFGPSVYbAaAt@w@|@_ALMnAoAd@_@TSHEDGdA}@RUZ]KSS[KQq@mAGICEYg@OWAAq@kAEGKSGMKOOWEIOKIGCCi@YGWU{@BGxC_Dd@}@DKHa@?[EMD]H]HUDW\\y@Xe@BGHMb@k@h@u@i@t@_AgBSs@Ou@i@h@e@l@sAxBk@PI@qA^y@TYHgAB_@@?e@@o@@}@@k@Ig@Iq@Oo@Um@CEAEQc@kAiCg@mAIScFyLCGEKkBmEKWAy@Nc@j@iA~CsFcAgB',
 'service': 0,
 'steps': [{'arrival': 0,
            'distance': 0,
            'duration': 0,
            'location': [11.1051, 46.2185],
            'type': 'start'},
           {'arrival': 141,
            'distance': 1267,
            'duration': 141,
            'job': 0,
            'location': [11.092028617858888, 46.2173172744161],
            'service': 0,
            'type': 'job',
            '